## Advent of code 2018 day 1-10
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv environment (which is set up with pypy3.10-v7.3.13-win64)
# to activate it from a git bash shell: source .venv/Scripts/activate

import collections
import itertools
import functools
import re
import copy
import math
import sys
import time
import json
import heapq
import bisect
import random
import sortedcontainers

import zio

In [ ]:
# version check and timestamp
# NB the timestamp supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}')

In [ ]:
# 2018 day 2
# start_ts=1701377203
# mv ~/Downloads/input* data_src/2018-day-2-input.txt
# big input file looks like: list of strings
# idea: part 1 parse as list, then per line / word count letter frequencies, then count words
#  which have a letter with a certain frequency, then multiply those two values
# part 2: totally unrelated to part 1 (?) generate each pair of words, count different letters in
#  the same position, collect common letters, if only one different that's the answer

def count_freqs_n(strs, n):
    '''return number of strings with a letter that occurs exactly n times'''
    cnt=0
    for s in strs:
        counter=collections.Counter()
        for c in s:
            counter[c]+=1
        for v in counter.values():
            if v==n:
                cnt+=1
                break
    return cnt

sample1=open('data_src/2018-day-2-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

# part 1
a=count_freqs_n(lines, 2)
b=count_freqs_n(lines, 3)
n=a*b
print(f'part 1: {n=}')

# part 2
for i, sa in enumerate(lines):
    for sb in lines[:i]:
        assert len(sa)==len(sb)
        common=''
        diffcnt=0
        for j in range(len(sa)):
            if sa[j]==sb[j]:
                common=common+sa[j]
            else:
                diffcnt+=1
        if diffcnt==1:
            print(f'part 2: {common=}')


In [ ]:
# 2018 day 1
# start_ts=1701341821
# mv ~/Downloads/input* data_src/2018-day-1-input.txt
# big input file looks like: pos. and negative numbers
# idea: part 1 parse then add, skipping leading + (wasn't actually needed)
# part 2: keep looping, keeping reached frequencies in a set

sample1=open('data_src/2018-day-1-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

# part 1
n=0
for s in lines:
    if s[0]=='+':
        n+=int(s[1:])
    else:
        n+=int(s)
print(f'part 1: {n=}')

# part 2
n=0
seenfreq=set()
stopped=False
while not stopped:
    for s in lines:
        if s[0]=='+':
            n+=int(s[1:])
        else:
            n+=int(s)
        if n in seenfreq:
            stopped=True
            break
        seenfreq.add(n)
print(f'part 2: {n=}')

In [ ]:
# TEMPLATE
# 2018 day 1
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2018-day-1-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample2='''

'''

sample1=open('data_src/2018-day-1-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
groups=zio.get_line_groups(sample1.splitlines(), nostrip=False)
data=[ int(s) for s in lines[0].split(',') ]
data=[ s.split() for s in lines ]
data=[ [cmd, int(num), 0] for cmd, num in data ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed